In [1]:
from banzhaf_dt import *
import pandas as pd
import numpy as np
import entropy_estimators
from sklearn.model_selection import train_test_split
import copy

columns = ['code_num','thickness','uofcsize','uofcshape','adhesion','secsize',\
		'bnuclei','chromatinb','nnucleoi','mitoses','output']
data = pd.read_csv('breast-cancer-wisconsin.data',names=columns)
data.drop(['code_num'],1,inplace=True)
data.replace('?',-99999, inplace=True)
data = data.astype(float)
data['output'] = data['output'].astype(int)
columns.pop(-1)
columns.pop(0)

X = data[columns]
y = data['output']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
training_data = copy.copy(X_train)
training_data['output'] = y_train
training_data = training_data.reset_index(drop=True)
training_data.head()

,thickness,uofcsize,uofcshape,adhesion,secsize,bnuclei,chromatinb,nnucleoi,mitoses,output
0,3.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,2
1,1.0,1.0,1.0,3.0,1.0,3.0,1.0,1.0,1.0,2
2,8.0,7.0,8.0,5.0,5.0,10.0,9.0,10.0,1.0,4
3,2.0,1.0,1.0,2.0,2.0,1.0,3.0,1.0,1.0,2
4,1.0,1.0,3.0,1.0,2.0,1.0,2.0,1.0,1.0,2


In [ ]:
columns.append('output')
update_cols(columns)
update_data(training_data[columns].values)
banzhaf_tree_untouched = build_tree(training_data[columns].values,banzhaf=True)
print_tree(banzhaf_tree_untouched)

In [ ]:
def select_kbest_features(k,data):
    output = list(data['output'].values.reshape(-1,1))
    S = []
    while k > 0:
        curr_info = 0
        MI = []
        columns = list(data.columns.values)
        for f in columns[:-1]:
            if len(S) == 0:
                y = list(data[f].values.reshape(-1,1))
                curr_info = ee.mi(output,y)
            else:
                if f in S:
                    continue
                else:
                    y = list(data[f].values.reshape(-1,1))
                    z = list(data[S].values.reshape(-1,len(S)))
                    curr_info = ee.cmi(output,y,z)
            if curr_info == 0:
                data.drop(f, axis=1, inplace=True)
            else:
                MI.append({'value':curr_info,'label':f})
        maxMIfeature = max(MI, key=lambda x:x['value'])
        S.append(maxMIfeature['label'])
        k = k - 1
    return S

In [ ]:
selected_feature = select_kbest_features(5,training_data)
selected_feature.append('output')
update_cols(selected_feature)
training_data = training_data[selected_feature].values
update_data(training_data)
banzhaf_tree = build_tree(training_data,banzhaf=True)
print_tree(banzhaf_tree)

In [ ]:
testing_data = np.column_stack((X_test,y_test))
prediction_banzhaf_untouched = []
prediction_banzhaf = []

for test_example in testing_data:
    prediction = classify(test_example,banzhaf_tree)
    prediction_banzhaf.append(prediction)
    prediction = classify(test_example,banzhaf_tree_untouched)
    prediction_banzhaf_untouched.append(prediction)

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,prediction_banzhaf_untouched))
print(accuracy_score(y_test,prediction_banzhaf))